In [1]:
## load SCE object
sce_R_filename = "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/all_da_d52_cells_sce.Rds"
sce = readRDS(sce_R_filename)
sce

Loading required package: SingleCellExperiment
Loading required package: SummarizedExperiment
Loading required package: GenomicRanges
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    

class: SingleCellExperiment 
dim: 32738 79801 
metadata(0):
assays(1): counts
rownames(32738): MIR1302-10 FAM138A ... AC002321.2 AC002321.1
rowData names(0):
colnames(79801): CCAGCGAGTTACTGAC-1-64 ACGCAGCGTTCGCTAA-1-45 ...
  CTTAGGAAGATTACCC-1-13 TCAACGACATAGAAAC-1-39
colData names(21): batch celltype ... treatment index
reducedDimNames(0):
spikeNames(0):
altExpNames(0):

In [2]:
### Filter SCE object
ncol(sce)
######
# consider only untreated cells
sce_filt = sce[,sce$treatment == 'NONE']
######
# consider only donors with > 10 cells
N=10
donors_keep = names(table(sce_filt$donor_id)[table(sce_filt$donor_id)>N])
sce_filt = sce_filt[, sce_filt$donor_id %in% donors_keep]  
ncol(sce_filt)

[1] 79801

Warning message in NSBS(i, x, exact = exact, strict.upper.bound = !allow.append, :
“subscript is an array, passing it thru as.vector() first”

[1] 54217

In [3]:
# filename = "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/da_d52_untreated_pseudocells_20_harmony_PCs_atleast10cells_10nn.csv"
# filename = "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/da_d52_untreated_pseudocells_20_harmony_PCs_atleast10cells_5nn.csv"
filename = "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/da_d52_untreated_pseudocells_20_harmony_PCs_atleast10cells_20nn.csv"
df_clusters = read.csv(filename, row.names = 1)

In [4]:
head(df_clusters)

cell,cluster,donor_id,ncells,pseudo_cell
CCAGCGAGTTACTGAC-1-64,cluster1,HPSI0513i-veve_2,88,HPSI0513i-veve_2-cluster1
GATCTAGCATCCAACA-1-65,cluster1,HPSI0513i-veve_2,88,HPSI0513i-veve_2-cluster1
TGGCCAGCAAGCTGTT-1-64,cluster1,HPSI0513i-veve_2,88,HPSI0513i-veve_2-cluster1
GCTGCGACAAGGACAC-1-64,cluster1,HPSI0513i-veve_2,88,HPSI0513i-veve_2-cluster1
ACATGGTGTTATCACG-1-64,cluster1,HPSI0513i-veve_2,88,HPSI0513i-veve_2-cluster1
GTTACAGCACAGAGGT-1-64,cluster1,HPSI0513i-veve_2,88,HPSI0513i-veve_2-cluster1


In [5]:
length(unique(df_clusters$pseudo_cell))

[1] 1013

In [6]:
length(unique(df_clusters$donor_id))

[1] 173

In [7]:
##### normalisation and log

In [8]:
sce_filt = scater::logNormCounts(sce_filt)
sce_filt

class: SingleCellExperiment 
dim: 32738 54217 
metadata(0):
assays(2): counts logcounts
rownames(32738): MIR1302-10 FAM138A ... AC002321.2 AC002321.1
rowData names(0):
colnames(54217): CCAGCGAGTTACTGAC-1-64 ACGTCAAGTCTAGCCG-1-51 ...
  CCCAATCCACAGGAGT-1-25 CTTAGGAAGATTACCC-1-13
colData names(21): batch celltype ... treatment index
reducedDimNames(0):
spikeNames(0):
altExpNames(0):

In [9]:
df = as.data.frame(as.matrix(logcounts(sce_filt)))
df[1:5,1:5]

,CCAGCGAGTTACTGAC-1-64,ACGTCAAGTCTAGCCG-1-51,CGTCTACGTCGGGTCT-1-35,TGTATTCAGGCAAAGA-1-25,GTTCATTAGTGGTAAT-1-73
MIR1302-10,0,0,0,0,0
FAM138A,0,0,0,0,0
OR4F5,0,0,0,0,0
RP11-34P13.7,0,0,0,0,0
RP11-34P13.8,0,0,0,0,0


In [10]:
df1 = t(df)
df1[1:5,1:5]

,MIR1302-10,FAM138A,OR4F5,RP11-34P13.7,RP11-34P13.8
CCAGCGAGTTACTGAC-1-64,0,0,0,0,0
ACGTCAAGTCTAGCCG-1-51,0,0,0,0,0
CGTCTACGTCGGGTCT-1-35,0,0,0,0,0
TGTATTCAGGCAAAGA-1-25,0,0,0,0,0
GTTCATTAGTGGTAAT-1-73,0,0,0,0,0


In [ ]:
df1$cell = rownames(df1)

Warning message in df1$cell = rownames(df1):
“Coercing LHS to a list”

In [ ]:
library(dplyr)

In [ ]:
df2 = inner_join(df_clusters, df1)
df2[1:5,(ncol(df2)-5):ncol(df2)]

Warning message in df1$cell = rownames(df1):
“Coercing LHS to a list”

In [28]:
##### (mean) aggregation

In [ ]:
mean = matrix(0, nrow = nrow(sce_filt), ncol = length(unique(df_clusters$pseudo_cell)))
for (i in 1:length(unique(df_clusters$pseudo_cell))){
    pseudocell = unique(df_clusters$pseudo_cell)[i]
    cells = df_clusters[df_clusters$pseudo_cell == pseudocell,"cell"]
    if (length(cells)==1){
        mean[,i] = as.matrix(logcounts(sce_filt))[,cells]
    }
    else{
        mat = as.matrix(logcounts(sce_filt))[,cells]
        mean[,i] = rowMeans(mat)
    }  
}

In [ ]:
mean[1:5,1:5]

In [ ]:
rownames(mean) = rownames(sce_filt)
colnames(mean) = unique(df_clusters$pseudo_cell)

In [ ]:
# write.csv(mean, "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/da_d52_untreated_pseudocells_20_harmony_PCs_atleast10cells_10nn_means.csv")
# write.csv(mean, "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/da_d52_untreated_pseudocells_20_harmony_PCs_atleast10cells_5nn_means.csv")
write.csv(mean, "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/da_d52_untreated_pseudocells_20_harmony_PCs_atleast10cells_20nn_means.csv")

In [ ]:
dim(mean)

In [ ]:
##### PCs computation

In [ ]:
pcs = prcomp(t(mean))
head(pcs$x)

In [ ]:
# write.csv(pcs$x[,1:500], "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/da_d52_untreated_pseudocells_20_harmony_PCs_atleast10cells_10nn_mean_500pcs.csv")
# write.csv(pcs$x[,1:500], "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/da_d52_untreated_pseudocells_20_harmony_PCs_atleast10cells_5nn_mean_500pcs.csv")
write.csv(pcs$x[,1:500], "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/da_d52_untreated_pseudocells_20_harmony_PCs_atleast10cells_20nn_mean_500pcs.csv")

In [ ]:
sample_df = data.frame(genotype_individual_id = gsub("-cluster.*","",rownames(pcs$x)),
                       phenotype_sample_id = rownames(pcs$x))
tail(sample_df)

In [ ]:
# write.csv(sample_df, "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/da_d52_untreated_pseudocells_20_harmony_PCs_atleast10cells_10nn_sample_mapping.csv", quote = F)
# write.csv(sample_df, "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/da_d52_untreated_pseudocells_20_harmony_PCs_atleast10cells_5nn_sample_mapping.csv", quote = F)
write.csv(sample_df, "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/da_d52_untreated_pseudocells_20_harmony_PCs_atleast10cells_20nn_sample_mapping.csv", quote = F)